# Sentiment-Driven Crypto Coin Report

## RSS feeds + LLM-powered sentiment analysis

This notebook builds on the Day 5 brochure generator pattern, but targets **crypto news** instead of company websites.

**Pipeline (3 LLM calls):**
1. Fetch recent headlines from CoinTelegraph and CoinDesk RSS feeds (no API key needed)
2. **LLM #1** — Filter headlines relevant to the given coin
3. Deep-scrape the top article pages for full content using `basic_website_scraper.py`
4. **LLM #2** — Clean up raw scraped text into concise article summaries
5. **LLM #3** — Generate a streaming sentiment report with emojis and trader-friendly tone

In [61]:
import os
import sys
import json
import xml.etree.ElementTree as ET
import urllib.request
from pathlib import Path
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

sys.path.insert(0, str(Path().resolve()))
from basic_website_scraper import fetch_js_website

load_dotenv(override=True)
api_key = os.getenv("OPENROUTER_API_KEY")

if api_key and len(api_key) > 10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please set OPENROUTER_API_KEY in your .env file!")

MODEL = "openai/gpt-4.1-mini"
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

API key looks good so far


In [62]:
COIN_TAG_FEEDS = {
    "bitcoin": "https://cointelegraph.com/rss/tag/bitcoin",
    "ethereum": "https://cointelegraph.com/rss/tag/ethereum",
    "solana": "https://cointelegraph.com/rss/tag/solana",
    "xrp": "https://cointelegraph.com/rss/tag/xrp",
}

GENERAL_FEEDS = [
    "https://cointelegraph.com/rss",
    "https://www.coindesk.com/arc/outboundfeeds/rss/?outputType=xml",
]


def _parse_rss_xml(xml_bytes: bytes) -> list[dict]:
    """Parse RSS XML and return a list of {'title', 'url', 'description', 'pubDate'} dicts."""
    root = ET.fromstring(xml_bytes)
    entries = []
    for item in root.iter("item"):
        title = (item.findtext("title") or "").strip()
        link = (item.findtext("link") or "").strip()
        desc = (item.findtext("description") or "").strip()
        pub_date = (item.findtext("pubDate") or "").strip()
        if title and link:
            entries.append({
                "title": title,
                "url": link,
                "description": desc[:300],
                "pubDate": pub_date,
            })
    return entries


def fetch_rss_entries(coin: str) -> list[dict]:
    """
    Fetch RSS entries relevant to a coin.
    Uses a coin-specific CoinTelegraph tag feed when available,
    otherwise falls back to the general CoinTelegraph + CoinDesk feeds.
    """
    urls = []
    tag_url = COIN_TAG_FEEDS.get(coin.lower())
    if tag_url:
        urls.append(tag_url)
    urls.extend(GENERAL_FEEDS)

    all_entries: list[dict] = []
    seen_urls: set[str] = set()

    for feed_url in urls:
        print(f"Fetching RSS: {feed_url}")
        try:
            req = urllib.request.Request(feed_url, headers={"User-Agent": "Mozilla/5.0"})
            with urllib.request.urlopen(req, timeout=15) as resp:
                xml_bytes = resp.read()
            for entry in _parse_rss_xml(xml_bytes):
                if entry["url"] not in seen_urls:
                    seen_urls.add(entry["url"])
                    all_entries.append(entry)
        except Exception as e:
            print(f"  Could not fetch {feed_url}: {e}")

    print(f"Collected {len(all_entries)} unique RSS entries across {len(urls)} feeds")
    return all_entries

In [63]:
filter_system_prompt = """You are given a JSON list of crypto news headlines.
Select ONLY the headlines that are directly relevant to the specified cryptocurrency.
Respond strictly in JSON with this schema:
{"selected": [{"title": "...", "url": "...", "description": "..."}]}
Return at most 10 headlines. If none are relevant, return {"selected": []}."""


def filter_headlines(entries: list[dict], coin: str) -> list[dict]:
    """Use the LLM to pick headlines relevant to a specific coin."""
    headlines_json = json.dumps(entries[:40], ensure_ascii=False)
    user_prompt = (
        f"Coin: {coin}\n\nHeadlines:\n{headlines_json[:4_000]}"
    )
    print(f"Asking {MODEL} to filter headlines for '{coin}'...")
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": filter_system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        response_format={"type": "json_object"},
    )
    result = json.loads(response.choices[0].message.content)
    selected = result.get("selected", [])
    print(f"Selected {len(selected)} relevant headlines")
    return selected

In [64]:
def deep_scrape_articles(headlines: list[dict], max_articles: int = 5, max_chars: int = 2_000) -> str:
    """
    Scrape the top articles by URL and return their combined text,
    each truncated to max_chars.
    """
    articles_with_urls = [h for h in headlines if h.get("url")]
    selected = articles_with_urls[:max_articles]
    print(f"Deep-scraping {len(selected)} articles...")

    combined = ""
    for i, headline in enumerate(selected, 1):
        title = headline["title"]
        url = headline["url"]
        print(f"  [{i}/{len(selected)}] {title}")
        try:
            text = fetch_js_website(url)
            combined += f"\n\n### Article: {title}\nSource: {url}\n\n{text[:max_chars]}"
        except Exception as e:
            print(f"    Skipped (error: {e})")
            combined += f"\n\n### Article: {title}\n(Could not scrape — {e})"

    return combined

In [65]:
cleanup_system_prompt = """You are a news editor preparing article content for a crypto analyst.
You receive raw scraped webpage text that contains the actual article mixed with
navigation menus, ads, cookie banners, related article links, social sharing buttons,
and other site chrome.

For EACH article section (marked by ### Article:), extract ONLY the core news content
and produce a clean, concise summary (3-5 sentences max). Keep key facts, numbers,
quotes, and dates. Remove all website junk.

Respond with the cleaned articles in this format:
### [Article Title]
[Clean 3-5 sentence summary]

(Repeat for each article.)"""


def cleanup_scraped_content(raw_content: str) -> str:
    """LLM call to strip website junk from scraped articles and produce clean summaries."""
    print(f"Asking {MODEL} to clean up scraped content...")
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": cleanup_system_prompt},
            {"role": "user", "content": raw_content[:6_000]},
        ],
    )
    cleaned = response.choices[0].message.content
    print(f"Cleanup done — {len(cleaned)} chars (was {len(raw_content)} chars raw)")
    return cleaned

In [66]:
report_system_prompt = """You're a crypto-native market analyst — think CoinBureau meets Crypto Twitter.
You talk to fellow traders, not Wall Street suits. Skip the jargon, keep it real.

Given cleaned-up news summaries about a specific coin, write a short sentiment report in markdown.

Format:
- Start with a one-line header using the coin name and an emoji for the vibe
  (🟢 bullish, 🔴 bearish, 🟡 neutral), e.g. "# Bitcoin 🟢 — Bulls Are Back in Town"
- TWO short paragraphs max.
- First paragraph: the overall mood and what's driving it. Name the key stories.
  Use emojis naturally (🚀 📈 📉 🐋 💰 🔥 ⚠️ etc.) but don't overdo it.
- Second paragraph: what to watch — key price levels, upcoming events, or risks.
  End with a punchy one-liner.
- Use unicode arrows (→ ↑ ↓), em dashes (—), and bullet-free flowing prose.
- NO financial advice. NO technical jargon like "derivatives skew" or "basis spread" —
  say it in plain trader talk instead.
- Keep sentences short and punchy. This is a quick briefing, not a research paper."""

In [67]:
def build_report_user_prompt(coin: str, article_content: str) -> str:
    """Assemble the user prompt for the report generation call."""
    prompt = (
        f"Generate a sentiment report for **{coin.upper()}** based on these recent news articles:\n\n"
        f"{article_content}"
    )
    return prompt[:5_000]

In [68]:
def stream_coin_report(coin: str):
    """
    Full pipeline: RSS -> filter -> deep-scrape -> cleanup -> stream report.
    Three LLM calls total. Prints all intermediate outputs.
    """
    separator = "=" * 72

    # Step 1: Fetch RSS entries
    entries = fetch_rss_entries(coin)
    if not entries:
        display(Markdown(f"**No RSS entries fetched.**"))
        return

    print(f"\n{separator}")
    print(f"RAW RSS ENTRIES ({len(entries)} total, showing first 5)")
    print(separator)
    for e in entries[:5]:
        print(f"  [{e.get('pubDate', 'no date')}]")
        print(f"  {e['title']}")
        print(f"  {e['url']}\n")

    # Step 2: LLM filters to relevant headlines (LLM call #1)
    headlines = filter_headlines(entries, coin)
    if not headlines:
        display(Markdown(f"**No headlines found relevant to {coin}.**"))
        return

    print(f"\n{separator}")
    print(f"FILTERED HEADLINES")
    print(separator)
    print(json.dumps(headlines, indent=2, ensure_ascii=False))

    # Step 3: Deep-scrape the top articles
    print(f"\n{separator}")
    print("DEEP-SCRAPING ARTICLES")
    print(separator)
    raw_article_content = deep_scrape_articles(headlines)

    print(f"\n{separator}")
    print("RAW SCRAPED CONTENT")
    print(separator)
    print(raw_article_content[:3_000])
    if len(raw_article_content) > 3_000:
        print(f"\n... (truncated, {len(raw_article_content)} chars total)")

    # Step 4: LLM cleans up the scraped content (LLM call #2)
    print(f"\n{separator}")
    print("CLEANING UP SCRAPED CONTENT")
    print(separator)
    cleaned_content = cleanup_scraped_content(raw_article_content)

    print(f"\n{separator}")
    print("CLEANED ARTICLE SUMMARIES")
    print(separator)
    print(cleaned_content)

    # Step 5: Stream the sentiment report (LLM call #3)
    user_prompt = build_report_user_prompt(coin, cleaned_content)
    print(f"\n{separator}")
    print(f"STREAMING SENTIMENT REPORT FOR {coin.upper()}")
    print(separator)

    stream = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": report_system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        stream=True,
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        update_display(Markdown(response), display_id=display_handle.display_id)

In [69]:
stream_coin_report("bitcoin")

Fetching RSS: https://cointelegraph.com/rss/tag/bitcoin


Fetching RSS: https://cointelegraph.com/rss
Fetching RSS: https://www.coindesk.com/arc/outboundfeeds/rss/?outputType=xml
Collected 85 unique RSS entries across 3 feeds

RAW RSS ENTRIES (85 total, showing first 5)
  [Wed, 25 Feb 2026 10:35:48 +0000]
  Bitcoin price climbs 3% as gold divergence signals ‘significant upside’
  https://cointelegraph.com/news/bitcoin-price-climbs-3-percent-gold-divergence-significant-upside?utm_source=rss_feed&utm_medium=rss_tag_bitcoin&utm_campaign=rss_partner_inbound

  [Wed, 25 Feb 2026 09:37:04 +0000]
  Bitcoin bounces to $66K as rumors swirl over Jane Street selling algorithm
  https://cointelegraph.com/news/bitcoin-bounces-66k-rumors-swirl-jane-street-selling-algorithm?utm_source=rss_feed&utm_medium=rss_tag_bitcoin&utm_campaign=rss_partner_inbound

  [Wed, 25 Feb 2026 09:09:00 +0000]
  Anchorage buys STRC as Wall Street shorts mount against Saylor’s Bitcoin proxy
  https://cointelegraph.com/news/anchorage-digital-holds-strategy-strc-most-shorted-stock?

# Bitcoin 🟡 — Flash Rally Meets Lingering Doubts

Bitcoin perks up with a fresh 3% pop, cruising back over $66K thanks to big ETF inflows and a rare divergence from gold that’s whispering about more upside ahead 🚀. The hype around tech and AI stocks is rubbing off, while institutional buyers like Fidelity and BlackRock are stepping back in after weeks of selling pressure. Yet the market’s still edgy, with rumors of Jane Street’s algorithmic selling stirring drama and keeping the order books thin — meaning every move feels amplified 📉🤏.

Keep an eye on whether BTC can hold above $66K as traders digest this rollercoaster. The ETF flows suggest more buyers lurking, but the lingering shadow of big Q4 dumps and wash trading rumors could spark sharp pullbacks. If Bitcoin can shake off the noise and prove this rebound’s solid, it might just be gearing up for a surprise comeback — but don’t blink, it’s still a razor’s edge game.

In [70]:
stream_coin_report("ethereum")

Fetching RSS: https://cointelegraph.com/rss/tag/ethereum
Fetching RSS: https://cointelegraph.com/rss
Fetching RSS: https://www.coindesk.com/arc/outboundfeeds/rss/?outputType=xml
Collected 85 unique RSS entries across 3 feeds

RAW RSS ENTRIES (85 total, showing first 5)
  [Tue, 24 Feb 2026 22:42:36 +0000]
  ETH options turn bearish as traders prepare for extended Ether price downside
  https://cointelegraph.com/news/eth-bounces-off-1-8k-as-multiple-ether-price-metrics-point-to-prolonged-weakness?utm_source=rss_feed&utm_medium=rss_tag_ethereum&utm_campaign=rss_partner_inbound

  [Tue, 24 Feb 2026 22:00:05 +0000]
  ESMA warns crypto perpetual derivatives likely fall under CFD rules
  https://cointelegraph.com/news/crypto-derivatives-eu-regulations-esma?utm_source=rss_feed&utm_medium=rss_tag_ethereum&utm_campaign=rss_partner_inbound

  [Tue, 24 Feb 2026 19:03:53 +0000]
  Longest Ether dip since 2022 ignored by whales: What’s next for ETH?
  https://cointelegraph.com/news/longest-ether-dip-

# Ethereum 🔴 — Bears Still Grip ETH as Pain Runs Deep

Ether’s been getting hit hard, dropping to $1,800 and wiping out over $220M in leveraged longs in just two days. Traders are loading up on puts, locking in downside protection with the put-to-call ratio spiking 2.2x. Whales are stepping back too, selling less on Binance and signaling a tighter liquidity squeeze around $2,000. With ETF outflows and ETH’s correlation to Bitcoin dragging it down, the mood is heavy and cautious.

Watch the $1,800 to $2,000 zone closely — it’s the battleground before any real bounce. Whales’ quieter moves and on-chain lows suggest sellers might still have the edge. If ETH can’t find support near these levels, the slide could linger longer. Bottom line — bears aren’t done yet, so strap in and keep your eyes on the charts.

In [71]:
stream_coin_report("solana")

Fetching RSS: https://cointelegraph.com/rss/tag/solana
Fetching RSS: https://cointelegraph.com/rss
Fetching RSS: https://www.coindesk.com/arc/outboundfeeds/rss/?outputType=xml
Collected 85 unique RSS entries across 3 feeds

RAW RSS ENTRIES (85 total, showing first 5)
  [Tue, 24 Feb 2026 02:51:01 +0000]
  3 Solana platforms to shutter following devastating $40M hack
  https://cointelegraph.com/news/step-finance-solanafloor-remora-markets-wind-down-operations?utm_source=rss_feed&utm_medium=rss_tag_solana&utm_campaign=rss_partner_inbound

  [Mon, 23 Feb 2026 18:36:00 +0000]
  Price predictions 2/23: SPX, DXY, BTC, ETH, XRP, BNB, SOL, DOGE, BCH, ADA
  https://cointelegraph.com/news/price-predictions-2-23-spx-dxy-btc-eth-xrp-bnb-sol-doge-bch-ada?utm_source=rss_feed&utm_medium=rss_tag_solana&utm_campaign=rss_partner_inbound

  [Mon, 23 Feb 2026 11:54:56 +0000]
  Crypto funds lose $288M as ETPs extend outflow run to five weeks
  https://cointelegraph.com/news/crypto-etp-five-week-exit-fresh-2

# Solana 🔴 — Tough Times After a Brutal $40M Hack

Solana’s vibe just took a hit with three key projects—Step Finance, SolanaFloor, and Remora Markets—shutting down after a massive $40M treasury hack hammered confidence. The fallout from Step Finance’s loss is still rippling, as they scramble to buy back tokens and compensate victims, but the damage is done. On top of that, macro fears from new global tariffs and a dip in BTC price are dragging all cryptos down, and Solana’s not immune to this sell-off storm 🌧️.

Watch for how SOL handles key support levels as market fear lingers, and keep an ear out for any fresh news on recoveries or new hack fallout. If Solana can dodge further technical wounds and catch a bounce, it could flip sentiment back up — but right now, heads are down. Solana’s got a rough road ahead, so buckle up and watch those charts closely ↓.

In [72]:
stream_coin_report("xrp")

Fetching RSS: https://cointelegraph.com/rss/tag/xrp
Fetching RSS: https://cointelegraph.com/rss
Fetching RSS: https://www.coindesk.com/arc/outboundfeeds/rss/?outputType=xml
Collected 85 unique RSS entries across 3 feeds

RAW RSS ENTRIES (85 total, showing first 5)
  [Mon, 23 Feb 2026 18:36:00 +0000]
  Price predictions 2/23: SPX, DXY, BTC, ETH, XRP, BNB, SOL, DOGE, BCH, ADA
  https://cointelegraph.com/news/price-predictions-2-23-spx-dxy-btc-eth-xrp-bnb-sol-doge-bch-ada?utm_source=rss_feed&utm_medium=rss_tag_xrp&utm_campaign=rss_partner_inbound

  [Mon, 23 Feb 2026 16:44:52 +0000]
  XRP price chart and whale activity warn of a drop below $1
  https://cointelegraph.com/news/xrp-price-chart-whale-activity-warn-drop-below-1-dollar?utm_source=rss_feed&utm_medium=rss_tag_xrp&utm_campaign=rss_partner_inbound

  [Mon, 23 Feb 2026 11:54:56 +0000]
  Crypto funds lose $288M as ETPs extend outflow run to five weeks
  https://cointelegraph.com/news/crypto-etp-five-week-exit-fresh-288-million-outflo

# XRP 🔴 — Bearish Clouds Gather Over $1

XRP’s looking shaky with a classic bear pennant flashing red 🚩—pointing down to around $0.80 if support cracks. Whale moves into Binance have traders spooked, adding fuel to the selling pressure. The broader crypto gloom isn’t helping either, as BTC and majors struggle and fear dominates the market vibe.

Keep an eye on the $1.22 level—it’s the last line before things could get uglier. If that breaks, expect a sharper drop. No clear signs of relief yet, so buckle up and watch those whales. XRP’s in the danger zone, and the bears smell blood.